# Run linear regression on the real (reparameterized) meshes

In [1]:
import src.setcwd

src.setcwd.main()

Working directory:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains/H2_SurfaceMatch
Directory added to path:  /home/adele/code/my28brains/src/notebooks
Directory added to path:  /home/adele/code/my28brains/src/notebooks/csv


In [2]:
import os
import numpy as np
import trimesh

import src.datasets.utils as data_utils
import src.preprocessing
import src.viz
import project_regression.default_config as default_config

INFO: Using pytorch backend


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
src.setcwd.main()
print(os.getcwd())

meshes_along_linear_regression_1_obbz3jta = np.load(
    "project_regression/notebooks/meshes_along_linear_regression_1_obbz3jta.npy"
)
reference_mesh = trimesh.load_mesh(
    r"notebooks/parameterized_mesh00_hormone_level0.63.ply.ply"
)

Working directory:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains/H2_SurfaceMatch
Directory added to path:  /home/adele/code/my28brains/src/notebooks
Directory added to path:  /home/adele/code/my28brains/src/notebooks/csv
/home/adele/code/my28brains/src


FileNotFoundError: [Errno 2] No such file or directory: 'project_regression/notebooks/meshes_along_linear_regression_1_obbz3jta.npy'

In [ ]:
meshes_along_linear_regression_1_obbz3jta.shape

In [ ]:
print(len(meshes_along_linear_regression_1_obbz3jta) / 6)
print(len(meshes_along_linear_regression_1_obbz3jta) / 10)

The number of "X" (progesterone levels) is either 10 or 6.

In [ ]:
from my28brains.H2_SurfaceMatch.utils.input_output import plotGeodesic

In [ ]:
geod = meshes_along_linear_regression_1_obbz3jta.reshape((10, -1, 3))
plotGeodesic(
    geod=geod,
    F=reference_mesh.faces,
    stepsize=7,
    file_name="notebooks/meshes_along_linear_regression_1_obbz3jta.ply",
)

In [ ]:
np.save("notebooks/mesh_test.npy", mesh_seq_to_plot)

In [ ]:
reference_mesh

In [ ]:
reference_mesh.faces.shape

In [ ]:
mesh = trimesh.Trimesh(
    vertices=meshes_along_linear_regression_1_obbz3jta, faces=reference_mesh.faces
)
mesh

In [ ]:
src.preprocessing.writing.trimesh_to_ply(
    mesh, "notebooks/meshes_along_linear_regression_1_obbz3jta.ply"
)

# Run linear regression from scratch

In [ ]:
mesh_list = []
mesh_dir = "results/meshes_parameterized"

for path in sorted(os.listdir(mesh_dir)):
    print(path)
    mesh = trimesh.load(os.path.join(mesh_dir, path))
    mesh_list.append(mesh)

In [ ]:
mesh0 = mesh_list[0]
mesh1 = mesh_list[1]

In [ ]:
mesh0

In [ ]:
mesh1

In [ ]:
np_list = [mesh.vertices for mesh in mesh_list]

In [ ]:
import numpy as np

np_seq = np.stack(np_list)

In [ ]:
np_seq.shape

In [ ]:
np_seq = np_seq.reshape((10, -1))

In [ ]:
import pandas as pd

DATA_PATH = os.path.join("../data", "hormones.csv")
df = pd.read_csv(DATA_PATH, delimiter=",")
print(f"df has length: {len(df)} days")
df[1:11]

In [ ]:
df["Prog"][1:9].values

In [ ]:
levels = df["Prog"][1:9].values.reshape((-1, 1))

In [ ]:
np_seq.shape
np_seq = np_seq[:-2]
np_seq.shape

In [ ]:
from sklearn.linear_model import LinearRegression

# Assuming X and y are already defined
X = levels.reshape(-1, 1)  # Reshape X to a 2D array if needed
y = np_seq

regressor = LinearRegression()
regressor.fit(X, y)

# Predicting values
y_pred = regressor.predict(X)

In [ ]:
y_pred = y_pred.reshape((10, -1, 3))
y = y.reshape((10, -1, 3))

# y_pred = y_pred.reshape((-1, 3))
# y = y.reshape((-1, 3))

In [ ]:
new_levels = np.arange(6.0, 16.0, step=1.0).reshape((-1, 1))
print(new_levels)
print(new_levels.shape)
y_pred_from_new_levels = regressor.predict(new_levels)
y_pred_from_new_levels = y_pred_from_new_levels.reshape((10, -1, 3))

In [ ]:
plotGeodesic(y, F=mesh0.faces, stepsize=4, file_name="notebooks/0710_mesh_sequence.ply")

In [ ]:
plotGeodesic(
    y_pred,
    F=mesh0.faces,
    stepsize=4,
    file_name="notebooks/0710_mesh_linear_regression.ply",
)

In [ ]:
plotGeodesic(
    y_pred_from_new_levels,
    F=mesh0.faces,
    stepsize=4,
    file_name="notebooks/0710_mesh_linear_prediction.ply",
)

In [ ]:
one_mesh = regressor.predict([[6.0]])
one_mesh = one_mesh.reshape((1, -1, 3))
print(one_mesh.shape)

In [ ]:
plotGeodesic(
    one_mesh,
    F=mesh0.faces,
    stepsize=4,
    file_name="notebooks/0710_one_mesh.ply",
)

In [ ]:
regressor.__dict__

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y, y_pred)

In [ ]:
y.shape

In [ ]:
y_pred.shape

In [ ]:
Adj_r2 = 1 - (1 - r2_score(y, y_pred)) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

In [ ]:
Adj_r2